<a href="https://colab.research.google.com/github/Abdallah-M-Ali/Machine-Learning-with-Python-Projects/blob/main/SMS_textual_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.8
!pip install tensorflow-gpu==2.8

In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
# !pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-05-16 14:55:54--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K   298KB/s    in 1.2s    

2022-05-16 14:56:00 (298 KB/s) - ‘train-data.tsv’ saved [358233/358233]

--2022-05-16 14:56:00--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[===========

In [3]:
def data_preprocessing(path, shuffle=True, BUFFER_SIZE=10000, BATCH_SIZE=64):
  dataset = pd.read_csv(path, sep='\t', names=['label', 'text'])
  dataset['label'] = dataset['label'].map({"ham": 1.0, 'spam': 0.0})
  dataset = tf.data.Dataset.from_tensor_slices((dataset['text'], dataset['label']))
  if shuffle:
    dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
  else:
    dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
  return dataset

In [4]:
train_dataset = data_preprocessing(train_file_path)
for text, label in train_dataset.take(1):
 print('texts: ', text.numpy()[:3])
 print('label: ', label.numpy()[:3])

texts:  [b'okay but i thought you were the expert'
 b"yeah i think my usual guy's still passed out from last night, if you get ahold of anybody let me know and i'll throw down"
 b"* am on a train back from northampton so i'm afraid not!"]
label:  [1. 1. 1.]


In [5]:
test_dataset = data_preprocessing(test_file_path, shuffle=False)
for text, label in test_dataset.take(1):
 print('texts: ', text.numpy()[:3])
 print('label: ', label.numpy()[:3])

texts:  [b'i am in hospital da. . i will return home in evening'
 b"not much, just some textin'. how bout you?"
 b"i probably won't eat at all today. i think i'm gonna pop. how was your weekend? did u miss me?"]
label:  [1. 1. 1.]


In [6]:
!pip install tensorflow

     |████████████████████████████████| 462 kB 18.3 MB/s 


In [8]:
VOCAB_SIZE=1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)

encoder.adapt(train_dataset.map(lambda text, label: text))
vocab = np.array(encoder.get_vocabulary())
print(vocab[:20])

['' '[UNK]' 'to' 'i' 'you' 'a' 'the' 'u' 'and' 'in' 'is' 'me' 'my' 'for'
 'your' 'of' 'it' 'call' 'have' 'on']


In [9]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])


In [10]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [11]:
history = model.fit(train_dataset, epochs=10,  validation_steps=30)
# validation_data=test_dataset,

Epoch 1/10
66/66 [==============================] - 14s 27ms/step - loss: 0.6672 - accuracy: 0.1340
Epoch 2/10
66/66 [==============================] - 2s 24ms/step - loss: 0.5368 - accuracy: 0.5092
Epoch 3/10
66/66 [==============================] - 1s 21ms/step - loss: 0.4033 - accuracy: 0.8428
Epoch 4/10
66/66 [==============================] - 2s 24ms/step - loss: 0.2532 - accuracy: 0.9220
Epoch 5/10
66/66 [==============================] - 2s 23ms/step - loss: 0.1425 - accuracy: 0.9713
Epoch 6/10
66/66 [==============================] - 2s 24ms/step - loss: 0.0923 - accuracy: 0.9789
Epoch 7/10
66/66 [==============================] - 2s 23ms/step - loss: 0.0730 - accuracy: 0.9821
Epoch 8/10
66/66 [==============================] - 2s 25ms/step - loss: 0.0597 - accuracy: 0.9856
Epoch 9/10
66/66 [==============================] - 3s 45ms/step - loss: 0.0513 - accuracy: 0.9864
Epoch 10/10
66/66 [==============================] - 1s 22ms/step - loss: 0.0432 - accuracy: 0.9885


In [12]:
test_loss, test_acc = model.evaluate(test_dataset)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

22/22 [==============================] - 3s 8ms/step - loss: 0.0632 - accuracy: 0.9842
Test Loss: 0.06315699964761734
Test Accuracy: 0.9841954112052917


In [13]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
      predictions = model.predict(np.array([pred_text]))[0][0]

      if round(predictions) >= 1:
          return [predictions, "ham"]
      return [predictions, "spam"]


  # return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[4.5831466, 'ham']


In [14]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()

You passed the challenge. Great job!
